In [1]:
from app.utils.t5 import *
import warnings
import gc
torch.cuda.empty_cache()
pl.seed_everything(42)
warnings.filterwarnings("ignore")
gc.collect()
torch.cuda.empty_cache()

Global seed set to 42


In [2]:
input_model_name = "d-t5-t5-base_grammar"
input_data_name = "preprocessed.csv"  

input_model_dir = "./Data/Model/"
input_data_dir = "./Data/Preprocessed/"
output_dir = "./Data/Generation/"
source_column = "outline"           
generated_column = "automatic_text"       
model_type = "t5"

input_data_path = "{}{}".format(input_data_dir, input_data_name)
new_data_path = "{}generated_{}.csv".format(output_dir,input_model_name)
input_model_path = "{}{}".format(input_model_dir,input_model_name)

In [3]:
data = pd.read_csv(input_data_path)

In [4]:
test = data[data.type == "test"]

In [5]:
model = SimpleT5()

In [6]:
input_model_path

'./Data/Model/d-t5-t5-base_grammar'

In [53]:
model.load_model(model_type, input_model_path, use_gpu = False) # input_model_path

In [70]:
text = "In this chapter? we first go explaining tf-idf then will get into neural nets and finally explain BERT."
input_text = ["grammar:" + text + " </s>"] # 

In [71]:
model.predict(input_text, batch_size = 1)[0]

  0%|          | 0/1 [00:00<?, ?it/s]

'In this chapter we first go explaining tf-idf then will get into neural nets and finally explain BERT.'

In [8]:
input_data = ("grammar: " + test[source_column]).to_list()
#input_data = test[source_column].to_list()
print(len(input_data))
automatic_lst = model.predict(input_data, batch_size = 15) # 30 15 4

45


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
new_data_path

'./Data/Generation/generated_d-t5-t5-large_grammar.csv'

In [10]:
test[generated_column] = automatic_lst
print(len(test))
test.to_csv(new_data_path, index=False)

45
